In [1]:
import os
from import_utils import ImportUtils

In [2]:
os.environ['db_host'] = ''
os.environ['db_name'] = ''
os.environ['db_user'] = ''
os.environ['db_pass'] = 'na'

In [3]:
iu = ImportUtils(os.environ["db_host"], os.environ["db_name"], os.environ["db_user"], os.environ["db_pass"])

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Setup
Assumes a local postgresql database is created with default postgres user
Assumes a local postgresql database called "work" is created and has postgis extension enabled

CREATE schema import;
CREATE schema property;

### Load the lookup table for King County

In [ ]:
import_utils.download_and_unzip_file("https://aqua.kingcounty.gov/extranet/assessor/Lookup.zip")
file_name = "data/EXTR_LookUp.csv"
table_name = "king_county_lookup"
import_utils.load_csv_to_postgres(table_name, file_name)
import_utils.run_clean_script("sql/king-county/clean_lookup.sql")
import_utils.run_sql("DROP TABLE import.%s" % table_name)
import_utils.run_command("rm %s" % file_name)

In [ ]:
!psql -U postgres -d work --tuples-only -c "select 'DROP TYPE IF EXISTS king_county_type_' || lu_type || ';  CREATE TYPE king_county_type_' || lu_type || ' AS ENUM (' || array_to_string(array_agg(quote_literal(descr)  ORDER BY descr), ', ') || ');' from (SELECT DISTINCT lu_type, descr FROM property.king_county_lookup) t group by lu_type;" > sql/create_king_county_enums.sql
!psql -U postgres -d work -f sql/create_king_county_enums.sql

### Load King County sales transactions

In [ ]:
iu.download_and_unzip_file("https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip")

In [4]:
file_name = "data/EXTR_RPSale.csv"
table_name = "king_county_sales"
iu.load_csv_to_postgres(table_name, file_name)

2020-12-24 10:21:55: start load_csv_to_postgres king_county_sales
2020-12-24 10:21:55: create table: import.king_county_sales
2020-12-24 10:21:55: running sql
2020-12-24 10:21:55: b'CREATE TABLE\n'
2020-12-24 10:21:55: done running sql
2020-12-24 10:21:55: load data
2020-12-24 10:23:02: b'COPY 2128280\n'
2020-12-24 10:23:02: trim data
2020-12-24 10:23:02: running sql
2020-12-24 10:23:22: b'UPDATE 2128280\n'
2020-12-24 10:23:22: done running sql
2020-12-24 10:23:22: empty to null
2020-12-24 10:23:22: running sql
2020-12-24 10:23:36: b'UPDATE 2128280\n'
2020-12-24 10:23:36: done running sql
2020-12-24 10:23:36: done load_csv_to_postgres import.king_county_sales


In [ ]:
# takes a bit over 1 minute to run
iu.run_clean_script("sql/king-county/clean_sales.sql")
iu.run_sql("DROP TABLE import.%s" % table_name)
iu.run_command("rm %s" % file_name)

In [ ]:
import_utils.run_clean_script("sql/king-county/clean_sales.sql")

In [ ]:
import_utils.download_and_unzip_file("https://aqua.kingcounty.gov/extranet/assessor/Residential%20Building.zip")
file_name = "data/EXTR_ResBldg.csv"
table_name = "king_county_building"
import_utils.load_csv_to_postgres(table_name, file_name)
import_utils.run_clean_script(table_name)
import_utils.run_sql("DROP TABLE import.%s" % table_name)
import_utils.run_command("rm %s" % file_name)

In [ ]:
import_utils.download_and_unzip_file("https://aqua.kingcounty.gov/extranet/assessor/Apartment%20Complex.zip")
file_name = "data/EXTR_AptComplex.csv"
table_name = "king_county_apt_complex"
import_utils.load_csv_to_postgres(table_name, file_name)
import_utils.run_clean_script(table_name)
import_utils.run_sql("DROP TABLE import.%s" % table_name)
import_utils.run_command("rm %s" % file_name)

In [ ]:
import_utils.download_and_unzip_file("https://aqua.kingcounty.gov/extranet/assessor/Commercial%20Building.zip")
file_name = "data/EXTR_CommBldg.csv"
table_name = "king_county_comm_bldg"
import_utils.load_csv_to_postgres(table_name, file_name)
import_utils.run_clean_script(table_name)
import_utils.run_sql("DROP TABLE import.%s" % table_name)
import_utils.run_command("rm EXTR_Comm*")

### King County Condos imports

In [ ]:
import_utils.download_and_unzip_file("https://aqua.kingcounty.gov/extranet/assessor/Condo%20Complex%20and%20Units.zip")

file_name = "data/EXTR_CondoUnit2.csv"
table_name = "king_county_condo_unit"
import_utils.load_csv_to_postgres(table_name, file_name)
import_utils.run_clean_script(table_name)
import_utils.run_sql("DROP TABLE import.%s" % table_name)

file_name = "data/EXTR_CondoComplex.csv"
table_name = "king_county_condo_complex"
import_utils.load_csv_to_postgres(table_name, file_name)
import_utils.run_clean_script(table_name)
import_utils.run_command("rm EXTR_Condo*")

### Two approaches to King County parcels

In [ ]:
import_utils.download_and_unzip_file("https://aqua.kingcounty.gov/extranet/assessor/Parcel.zip")
file_name = "data/EXTR_Parcel.csv"
table_name = "king_county_parcel"
import_utils.load_csv_to_postgres(table_name, file_name)
import_utils.run_clean_script(table_name)
import_utils.run_sql("DROP TABLE import.%s" % table_name)
import_utils.run_command("rm %s" % file_name)

In [ ]:
import_utils.download_and_unzip_file("ftp://ftp.kingcounty.gov/gis-web/GISData/parcel_address.zip")

In [ ]:
!shp2pgsql -s 2285 -W latin1 -D data/parcel_address/parcel_address.shp import.king_county_parcel_geo > data/king_county_parcel_geo.sql
!psql -U postgres -d work -f data/king_county_parcel_geo.sql

In [ ]:
import_utils.run_clean_script("king_county_parcel_geo")
import_utils.run_command("rm data/king_county_parcel_geo.sql")
import_utils.run_command("rm -rf data/parcel_address/")